In [4]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\aniratho\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         860 KB

The following NEW packages will be INSTALLED:

  alt

In [5]:
# Import
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
print('Libraries imported.')


Libraries imported.


In [6]:
# Source Wikipedia URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Parse to html
html_parsed_result = BeautifulSoup(requests.get(url).text, 'html.parser')

# Parse wikipedia table
nb_tb = html_parsed_result.find('table', class_ = 'wikitable')

# Find rows
nb_rw = nb_tb.find_all('tr')

# extract 'Postcode', 'Borough', 'Neighbourhood' from wikipedia table
nb_info = []

# Append data to nb_info list
for row in nb_rw:
    info = row.text.split('\n')[1:-1] # remove empty str (first and last items)
    nb_info.append(info)    
nb_info[0][-1] = 'Neighborhood'

# Create dataframe from nb_info list
nb_df = pd.DataFrame(nb_info[1:], columns=nb_info[0])

In [7]:
# Clean by replacing not assigned values in Neighborhood column to the same as Borough in the cell
nb_not_assigned = nb_df.index[nb_df['Neighborhood'] == 'Not assigned']
for i in nb_not_assigned:
    nb_df['Neighborhood'][i] = nb_df['Borough'][i]

C:\Users\aniratho\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
# Clean by droping unassigned cells
boroughs_not_assigned = nb_df.index[nb_df['Borough'] == 'Not assigned']
neighborhoods_not_assigned = nb_df.index[nb_df['Neighborhood'] == 'Not assigned']
neighborhoods_borough_not_assigned = boroughs_not_assigned & neighborhoods_not_assigned
nb_df.drop(nb_df.index[boroughs_not_assigned], inplace=True)
nb_df.reset_index(drop=True, inplace=True)

# Print top 10
nb_df.head(12)

,Postal Code,,Borough,,Neighborhood
0,M3A,,North York,,Parkwoods
1,M4A,,North York,,Victoria Village
2,M5A,,Downtown Toronto,,"Regent Park, Harbourfront"
3,M6A,,North York,,"Lawrence Manor, Lawrence Heights"
4,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government"
5,M9A,,Etobicoke,,"Islington Avenue, Humber Valley Village"
6,M1B,,Scarborough,,"Malvern, Rouge"
7,M3B,,North York,,Don Mills
8,M4B,,East York,,"Parkview Hill, Woodbine Gardens"
9,M5B,,Downtown Toronto,,"Garden District, Ryerson"


In [12]:
# Group same borough
gp = nb_df.groupby('Postal Code')
gp_nb = gp['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
gp_boroughs = gp['Borough'].apply(lambda x: set(x).pop())
gp_df = pd.DataFrame(list(zip(gp_boroughs.index, gp_boroughs, gp_nb)))
gp_df.columns = ['Postal Code', 'Borough', 'Neighborhood']


# Print top 10
gp_df.head(12)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [13]:
print('The DataFrame shape is', gp_df.shape)

The DataFrame shape is (103, 3)
